In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
branch_info_df = pd.read_csv("../data/source_files/tpl-branch-general-information-2023.csv")
branch_info_df.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   112 non-null    int64  
 1   BranchCode            112 non-null    object 
 2   PhysicalBranch        112 non-null    int64  
 3   BranchName            112 non-null    object 
 4   Address               103 non-null    object 
 5   PostalCode            103 non-null    object 
 6   Website               108 non-null    object 
 7   Telephone             105 non-null    object 
 8   SquareFootage         110 non-null    object 
 9   PublicParking         100 non-null    object 
 10  KidsStop              100 non-null    float64
 11  LeadingReading        100 non-null    float64
 12  CLC                   100 non-null    float64
 13  DIH                   100 non-null    float64
 14  TeenCouncil           100 non-null    float64
 15  YouthHub              1

In [3]:
# Remove columns. Rename columns for better clarity. Remove branches with no physical location.

branch_info_df = branch_info_df.drop(columns=['_id','Address','PostalCode','Website','Telephone','WardNo','WardName'],axis=1)
branch_info_df.rename(columns={'CLC':'ComputerLearningCentre', 'DIH':'DigitalInnovationHub', 'Lat': 'Latitude', 'Long': 'Longitude'}, inplace=True)
branch_info_df = branch_info_df[branch_info_df['PhysicalBranch'] != 0]
branch_info_df['NBHDName'] = branch_info_df['NBHDName'].str.strip()
branch_info_df[['SquareFootage','KidsStop','LeadingReading','ComputerLearningCentre','DigitalInnovationHub','TeenCouncil','YouthHub','AdultLiteracyProgram','Workstations','NBHDNo','TPLNIA','PresentSiteYear']] = branch_info_df[['SquareFootage','KidsStop','LeadingReading','ComputerLearningCentre','DigitalInnovationHub','TeenCouncil','YouthHub','AdultLiteracyProgram','Workstations','NBHDNo','TPLNIA','PresentSiteYear']].astype('int64')
branch_info_df['No_of_Programs'] = branch_info_df[['KidsStop', 'LeadingReading', 'ComputerLearningCentre', 'DigitalInnovationHub', 'TeenCouncil', 'YouthHub', 'AdultLiteracyProgram']].sum(axis=1)

'''Discovered a discrepancy for 'NBHDNo' and 'NBHDName'between branch information dataset and neighbourhood profile 
source datasets.Branch information file had NBHDNo values 14 and 26 which do not exist. Three libraries had were assigned to these
neighbourhoods: BranchCode = BC, DO, EA. The library address in the source file was used to identify the correct 
NBHDNo and NBHDName those libraries'''

branch_info_df.loc[branch_info_df['BranchCode'] == 'BC', ['NBHDNo', 'NBHDName']] = 154, 'Oakdale-Beverley Heights'
branch_info_df.loc[branch_info_df['BranchCode'] == 'DO', ['NBHDNo', 'NBHDName']] = 155, 'Downsview'
branch_info_df.loc[branch_info_df['BranchCode'] == 'EA', ['NBHDNo', 'NBHDName']] = 158, 'Islington'

branch_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 111
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BranchCode              100 non-null    object 
 1   PhysicalBranch          100 non-null    int64  
 2   BranchName              100 non-null    object 
 3   SquareFootage           100 non-null    int64  
 4   PublicParking           100 non-null    object 
 5   KidsStop                100 non-null    int64  
 6   LeadingReading          100 non-null    int64  
 7   ComputerLearningCentre  100 non-null    int64  
 8   DigitalInnovationHub    100 non-null    int64  
 9   TeenCouncil             100 non-null    int64  
 10  YouthHub                100 non-null    int64  
 11  AdultLiteracyProgram    100 non-null    int64  
 12  Workstations            100 non-null    int64  
 13  ServiceTier             100 non-null    object 
 14  Latitude                100 non-null    float64

In [4]:
branch_info_df.head()

,BranchCode,PhysicalBranch,BranchName,SquareFootage,PublicParking,KidsStop,LeadingReading,ComputerLearningCentre,DigitalInnovationHub,TeenCouncil,...,AdultLiteracyProgram,Workstations,ServiceTier,Latitude,Longitude,NBHDNo,NBHDName,TPLNIA,PresentSiteYear,No_of_Programs
0,AB,1,Albion,29000,59,1,1,1,1,1,...,1,38,DL,43.739826,-79.584096,2,Mount Olive-Silverstone-Jamestown,1,2017,7
1,ACD,1,Albert Campbell,28957,45,0,1,1,1,1,...,0,36,DL,43.708019,-79.269252,120,Clairlea-Birchmount,1,1971,5
2,AD,1,Alderwood,7341,shared,0,0,0,0,0,...,0,7,NL,43.601944,-79.547252,20,Alderwood,0,1999,0
3,AG,1,Agincourt,27000,86,0,1,1,1,0,...,0,42,DL,43.785167,-79.293430,118,Tam O'Shanter-Sullivan,0,1991,4
4,AH,1,Armour Heights,2988,shared,0,0,0,0,0,...,0,5,NL,43.739337,-79.421889,39,Bedford Park-Nortown,0,1982,0


In [5]:
# Convert 'NBHDNo' to string and add leading zeros
branch_info_df['NBHDNo'] = branch_info_df['NBHDNo'].apply(lambda x: str(x).zfill(3))
branch_info_df

,BranchCode,PhysicalBranch,BranchName,SquareFootage,PublicParking,KidsStop,LeadingReading,ComputerLearningCentre,DigitalInnovationHub,TeenCouncil,...,AdultLiteracyProgram,Workstations,ServiceTier,Latitude,Longitude,NBHDNo,NBHDName,TPLNIA,PresentSiteYear,No_of_Programs
0,AB,1,Albion,29000,59,1,1,1,1,1,...,1,38,DL,43.739826,-79.584096,002,Mount Olive-Silverstone-Jamestown,1,2017,7
1,ACD,1,Albert Campbell,28957,45,0,1,1,1,1,...,0,36,DL,43.708019,-79.269252,120,Clairlea-Birchmount,1,1971,5
2,AD,1,Alderwood,7341,shared,0,0,0,0,0,...,0,7,NL,43.601944,-79.547252,020,Alderwood,0,1999,0
3,AG,1,Agincourt,27000,86,0,1,1,1,0,...,0,42,DL,43.785167,-79.293430,118,Tam O'Shanter-Sullivan,0,1991,4
4,AH,1,Armour Heights,2988,shared,0,0,0,0,0,...,0,5,NL,43.739337,-79.421889,039,Bedford Park-Nortown,0,1982,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,WP,1,Woodview Park,4110,shared,0,0,0,0,1,...,0,9,NL,43.739722,-79.538941,022,Humbermede,1,1964,1
108,WS,1,Woodside Square,9944,shared,0,1,0,0,1,...,0,22,NL,43.809463,-79.269548,129,Agincourt North,0,1977,2
109,WY,1,Wychwood,15798,0,0,0,0,0,0,...,0,11,NL,43.682181,-79.417548,096,Casa Loma,0,1916,0
110,YO,1,Yorkville,9053,0,0,0,0,0,0,...,0,7,NL,43.671826,-79.388644,095,Annex,0,1907,0


In [6]:
branch_info_df.to_csv('../data/cleaned_files/tpl_branch_general_info_clean.csv', index=False)

In [7]:
branch_stats_df = branch_info_df.copy()

### Branch Visits

In [8]:
branch_visits = pd.read_csv("../data/source_files/tpl-visits-annual-by-branch-2012-2022.csv")
branch_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129 entries, 0 to 1128
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         1129 non-null   int64 
 1   Year        1129 non-null   int64 
 2   BranchCode  1129 non-null   object
 3   Visits      1129 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 35.4+ KB


In [9]:
# Remove '_id' as it is not related to library data. 'BranchCode' is unique to each branch. 
branch_visits = branch_visits.drop('_id', axis=1)

# Remove branch codes which have no physical locations (these were in branch_info_df with PhysicalBranch=0)
branch_visits = branch_visits[~branch_visits['BranchCode'].isin(['AL', 'BKONE', 'BKTWO','DS','HLS','IL','LD','ME','OS','PR','SB','VIR'])]

branch_visits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1085 entries, 0 to 1128
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        1085 non-null   int64 
 1   BranchCode  1085 non-null   object
 2   Visits      1085 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 33.9+ KB


In [10]:
''' Based on data review, the blanks exist for periods where a library branch was closed to the public for 
repairs or renovations. Setting NaN to 0. '''
branch_visits = branch_visits.fillna(0)

branch_visits = pd.merge(branch_visits, branch_stats_df[['BranchCode', 'BranchName', 'NBHDNo', 'NBHDName']], on='BranchCode', how='left')

branch_visits = branch_visits[['NBHDNo', 'NBHDName', 'BranchCode', 'BranchName', 'Year', 'Visits']]

branch_visits

,NBHDNo,NBHDName,BranchCode,BranchName,Year,Visits
0,002,Mount Olive-Silverstone-Jamestown,AB,Albion,2012,522309
1,120,Clairlea-Birchmount,ACD,Albert Campbell,2012,214076
2,020,Alderwood,AD,Alderwood,2012,88065
3,118,Tam O'Shanter-Sullivan,AG,Agincourt,2012,434320
4,039,Bedford Park-Nortown,AH,Armour Heights,2012,67184
...,...,...,...,...,...,...
1080,022,Humbermede,WP,Woodview Park,2022,28304
1081,129,Agincourt North,WS,Woodside Square,2022,163061
1082,096,Casa Loma,WY,Wychwood,2022,37001
1083,095,Annex,YO,Yorkville,2022,75390


In [11]:
# Saving csv file for all years
branch_visits.to_csv('../data/cleaned_files/tpl_branch_visits_2012_2022_clean.csv', index=False)

### Branch Registrations

In [12]:
branch_regs = pd.read_csv("../data/source_files/tpl-card-registrations-annual-by-branch-2012-2022.csv")
branch_regs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id            1169 non-null   int64 
 1   Year           1169 non-null   int64 
 2   BranchCode     1169 non-null   object
 3   Registrations  1169 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 36.7+ KB


In [13]:
# Remove '_id' as it is not related to library data. 'BranchCode' is unique to each branch. 
branch_regs = branch_regs.drop('_id', axis=1)

# Remove branch codes which have no physical locations (these were in branch_info_df with PhysicalBranch=0)
branch_regs = branch_regs[~branch_regs['BranchCode'].isin(['AL', 'BKONE', 'BKTWO','DS','HLS','IL','LD','ME','OS','PR','SB','VIR'])]

branch_regs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1095 entries, 0 to 1168
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           1095 non-null   int64 
 1   BranchCode     1095 non-null   object
 2   Registrations  1095 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 34.2+ KB


In [14]:
''' Based on data review, the blanks exist for periods where a library branch was closed to the public for 
repairs or renovations. Setting NaN to 0. '''
branch_regs = branch_regs.fillna(0)

branch_regs = pd.merge(branch_regs, branch_stats_df[['BranchCode', 'BranchName','NBHDNo', 'NBHDName']], on='BranchCode', how='left')

branch_regs = branch_regs[['NBHDNo', 'NBHDName', 'BranchCode', 'BranchName', 'Year', 'Registrations']]

branch_regs

,NBHDNo,NBHDName,BranchCode,BranchName,Year,Registrations
0,002,Mount Olive-Silverstone-Jamestown,AB,Albion,2012,4939
1,120,Clairlea-Birchmount,ACD,Albert Campbell,2012,1695
2,020,Alderwood,AD,Alderwood,2012,445
3,118,Tam O'Shanter-Sullivan,AG,Agincourt,2012,3166
4,039,Bedford Park-Nortown,AH,Armour Heights,2012,467
...,...,...,...,...,...,...
1090,022,Humbermede,WP,Woodview Park,2022,413
1091,129,Agincourt North,WS,Woodside Square,2022,1788
1092,096,Casa Loma,WY,Wychwood,2022,1035
1093,095,Annex,YO,Yorkville,2022,1081


In [15]:
# Saving csv file for all years
branch_regs.to_csv('../data/cleaned_files/tpl_branch_registrations_2012_2022_clean.csv', index=False)

### Branch Circulation

In [16]:
branch_circ = pd.read_csv("../data/source_files/tpl-circulation-annual-by-branch-2012-2022.csv")
branch_circ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          1225 non-null   int64 
 1   Year         1225 non-null   int64 
 2   BranchCode   1225 non-null   object
 3   Circulation  1225 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 38.4+ KB


In [17]:
# Remove '_id' as it is not related to library data. 'BranchCode' is unique to each branch. 
branch_circ = branch_circ.drop('_id', axis=1)

# Remove branch codes which have no physical locations (these were in branch_info_df with PhysicalBranch=0)
branch_circ = branch_circ[~branch_circ['BranchCode'].isin(['AL', 'BKONE', 'BKTWO','DS','HLS','IL','LD','ME','OS','PR','SB','VIR'])]

branch_circ.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1095 entries, 0 to 1224
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         1095 non-null   int64 
 1   BranchCode   1095 non-null   object
 2   Circulation  1095 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 34.2+ KB


In [18]:
''' Based on data review, the blanks exist for periods where a library branch was closed to the public for 
repairs or renovations. Setting NaN to 0. '''
branch_circ = branch_circ.fillna(0)

branch_circ = pd.merge(branch_circ, branch_stats_df[['BranchCode', 'BranchName','NBHDNo', 'NBHDName']], on='BranchCode', how='left')

branch_circ = branch_circ[['NBHDNo', 'NBHDName', 'BranchCode', 'BranchName','Year', 'Circulation']]

branch_circ

,NBHDNo,NBHDName,BranchCode,BranchName,Year,Circulation
0,002,Mount Olive-Silverstone-Jamestown,AB,Albion,2012,515235
1,120,Clairlea-Birchmount,ACD,Albert Campbell,2012,446623
2,020,Alderwood,AD,Alderwood,2012,171050
3,118,Tam O'Shanter-Sullivan,AG,Agincourt,2012,1061437
4,039,Bedford Park-Nortown,AH,Armour Heights,2012,204490
...,...,...,...,...,...,...
1090,022,Humbermede,WP,Woodview Park,2022,35084
1091,129,Agincourt North,WS,Woodside Square,2022,176330
1092,096,Casa Loma,WY,Wychwood,2022,47953
1093,095,Annex,YO,Yorkville,2022,142124


In [19]:
# Saving csv file for all years
branch_circ.to_csv('../data/cleaned_files/tpl_branch_circulation_2012_2022_clean.csv', index=False)

### Branch Workstation Usage

In [20]:
branch_wrkstn_use_1 = pd.read_csv("../data/source_files/tpl-workstation-usage-annual-by-branch-2012-2017.csv")
branch_wrkstn_use_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         598 non-null    int64 
 1   Year        598 non-null    int64 
 2   BranchCode  598 non-null    object
 3   Sessions    598 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 18.8+ KB


In [21]:
branch_wrkstn_use_2 = pd.read_csv("../data/source_files/tpl-workstation-usage-annual-by-branch-2018-2022.csv")
branch_wrkstn_use_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         492 non-null    int64 
 1   Year        492 non-null    int64 
 2   BranchCode  492 non-null    object
 3   Sessions    492 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 15.5+ KB


In [22]:
branch_wrkstn_all = pd.concat([branch_wrkstn_use_1, branch_wrkstn_use_2])

In [23]:
# Remove '_id' as it is not related to library data. 'BranchCode' is unique to each branch. 
branch_wrkstn_all = branch_wrkstn_all.drop('_id', axis=1)

# Remove branch codes which have no physical locations (these were in branch_info_df with PhysicalBranch=0)
branch_wrkstn_all = branch_wrkstn_all[~branch_wrkstn_all['BranchCode'].isin(['AL', 'BKONE', 'BKTWO','DS','HLS','IL','LD','ME','OS','PR','SB','VIR'])]

branch_wrkstn_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1081 entries, 0 to 491
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        1081 non-null   int64 
 1   BranchCode  1081 non-null   object
 2   Sessions    1081 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 33.8+ KB


In [24]:
''' Based on data review, the blanks exist for periods where a library branch was closed to the public for 
repairs or renovations. Setting NaN to 0. '''
branch_wrkstn_all = branch_wrkstn_all.fillna(0)

branch_wrkstn_all = pd.merge(branch_wrkstn_all, branch_stats_df[['BranchCode', 'BranchName','NBHDNo', 'NBHDName']], on='BranchCode', how='left')

branch_wrkstn_all = branch_wrkstn_all[['NBHDNo', 'NBHDName', 'BranchCode', 'BranchName','Year', 'Sessions']]

branch_wrkstn_all

,NBHDNo,NBHDName,BranchCode,BranchName,Year,Sessions
0,002,Mount Olive-Silverstone-Jamestown,AB,Albion,2012,132024
1,120,Clairlea-Birchmount,ACD,Albert Campbell,2012,131353
2,020,Alderwood,AD,Alderwood,2012,16279
3,118,Tam O'Shanter-Sullivan,AG,Agincourt,2012,140407
4,039,Bedford Park-Nortown,AH,Armour Heights,2012,11895
...,...,...,...,...,...,...
1076,022,Humbermede,WP,Woodview Park,2022,4115
1077,129,Agincourt North,WS,Woodside Square,2022,21028
1078,096,Casa Loma,WY,Wychwood,2022,3836
1079,095,Annex,YO,Yorkville,2022,8755


In [25]:
# Saving csv file for all years
branch_wrkstn_all.to_csv('../data/cleaned_files/tpl_branch_workstation_use_2012_2022_clean.csv', index=False)

### Neighbourhood Crime Rates

In [26]:
nbhd_crime = pd.read_csv("../data/source_files/neighbourhood-crime-rates - 2952.csv")
nbhd_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Columns: 185 entries, _id to geometry
dtypes: float64(127), int64(56), object(2)
memory usage: 228.5+ KB


In [27]:
nbhd_crime = nbhd_crime.loc[nbhd_crime['_id'] <= 158]

# Remove '_id' as it is not related to library data. 'BranchCode' is unique to each branch. 
nbhd_crime = nbhd_crime.drop(columns=['_id', 'geometry'],axis=1)

nbhd_crime.rename(columns={'AREA_NAME':'NBHDName', 'HOOD_ID':'NBHDNo'}, inplace=True)

nbhd_crime = nbhd_crime.drop(columns=['ASSAULT_2014','ASSAULT_2015','ASSAULT_2016','ASSAULT_2017','ASSAULT_2018','ASSAULT_2019','ASSAULT_2020','ASSAULT_2021','ASSAULT_2022','ASSAULT_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['AUTOTHEFT_2014','AUTOTHEFT_2015','AUTOTHEFT_2016','AUTOTHEFT_2017','AUTOTHEFT_2018','AUTOTHEFT_2019','AUTOTHEFT_2020','AUTOTHEFT_2021','AUTOTHEFT_2022','AUTOTHEFT_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['BIKETHEFT_2014','BIKETHEFT_2015','BIKETHEFT_2016','BIKETHEFT_2017','BIKETHEFT_2018','BIKETHEFT_2019','BIKETHEFT_2020','BIKETHEFT_2021','BIKETHEFT_2022','BIKETHEFT_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['BREAKENTER_2014','BREAKENTER_2015','BREAKENTER_2016','BREAKENTER_2017','BREAKENTER_2018','BREAKENTER_2019','BREAKENTER_2020','BREAKENTER_2021','BREAKENTER_2022','BREAKENTER_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['HOMICIDE_2014','HOMICIDE_2015','HOMICIDE_2016','HOMICIDE_2017','HOMICIDE_2018','HOMICIDE_2019','HOMICIDE_2020','HOMICIDE_2021','HOMICIDE_2022','HOMICIDE_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['ROBBERY_2014','ROBBERY_2015','ROBBERY_2016','ROBBERY_2017','ROBBERY_2018','ROBBERY_2019','ROBBERY_2020','ROBBERY_2021','ROBBERY_2022','ROBBERY_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['SHOOTING_2014','SHOOTING_2015','SHOOTING_2016','SHOOTING_2017','SHOOTING_2018','SHOOTING_2019','SHOOTING_2020','SHOOTING_2021','SHOOTING_2022','SHOOTING_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['THEFTFROMMV_2014','THEFTFROMMV_2015','THEFTFROMMV_2016','THEFTFROMMV_2017','THEFTFROMMV_2018','THEFTFROMMV_2019','THEFTFROMMV_2020','THEFTFROMMV_2021','THEFTFROMMV_2022','THEFTFROMMV_2023'],axis=1)

nbhd_crime = nbhd_crime.drop(columns=['THEFTOVER_2014','THEFTOVER_2015','THEFTOVER_2016','THEFTOVER_2017','THEFTOVER_2018','THEFTOVER_2019','THEFTOVER_2020','THEFTOVER_2021','THEFTOVER_2022','THEFTOVER_2023'],axis=1)

nbhd_crime = nbhd_crime.fillna(0)

nbhd_crime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158 entries, 0 to 157
Data columns (total 93 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NBHDName               158 non-null    object 
 1   NBHDNo                 158 non-null    int64  
 2   POPULATION_2023        158 non-null    int64  
 3   ASSAULT_RATE_2014      158 non-null    float64
 4   ASSAULT_RATE_2015      158 non-null    float64
 5   ASSAULT_RATE_2016      158 non-null    float64
 6   ASSAULT_RATE_2017      158 non-null    float64
 7   ASSAULT_RATE_2018      158 non-null    float64
 8   ASSAULT_RATE_2019      158 non-null    float64
 9   ASSAULT_RATE_2020      158 non-null    float64
 10  ASSAULT_RATE_2021      158 non-null    float64
 11  ASSAULT_RATE_2022      158 non-null    float64
 12  ASSAULT_RATE_2023      158 non-null    float64
 13  AUTOTHEFT_RATE_2014    158 non-null    float64
 14  AUTOTHEFT_RATE_2015    158 non-null    float64
 15  AUTOTHEFT_R

In [28]:
# Saving csv file for all years
nbhd_crime.to_csv('../data/cleaned_files/neighbourhood_crime_2014_2023_clean.csv', index=False)

In [29]:
df =  nbhd_crime.copy()

In [30]:
# Step 1: Initialize an empty DataFrame
df_new = pd.DataFrame(columns=['NBHDName', 'NBHDNo', 'POPULATION_2023', 'Year', 'Type_of_Crime', 'Rate_of_Crime'])

# Step 2 & 3: Iterate over columns and append rows to df_new
for _, row in df.iterrows():
    for col in df.columns[3:]:
        # Split column name on '_'
        split_col = col.rsplit('_', 1) # Split from the right once
        crime_type, year = split_col[0], split_col[1]
        
        # Further split crime_type on '_' to remove 'RATE'
        crime_type = crime_type.split('_')[0]
        
        # Create a new DataFrame for the current row and append it to df_new
        new_row = pd.DataFrame({
            'NBHDName': [row['NBHDName']],
            'NBHDNo': [row['NBHDNo']],
            'POPULATION_2023': [row['POPULATION_2023']],
            'Year': [year],
            'Type_of_Crime': [crime_type],
            'Rate_of_Crime': [row[col]]
        })
        df_new = pd.concat([df_new, new_row], ignore_index=True)

df_new = df_new[['NBHDNo', 'NBHDName', 'POPULATION_2023', 'Year', 'Type_of_Crime', 'Rate_of_Crime']]

# Convert 'NBHDNo' to string and add leading zeros
df_new['NBHDNo'] = df_new['NBHDNo'].apply(lambda x: str(x).zfill(3))

df_new

,NBHDNo,NBHDName,POPULATION_2023,Year,Type_of_Crime,Rate_of_Crime
0,174,South Eglinton-Davisville,21987,2014,ASSAULT,344.978638
1,174,South Eglinton-Davisville,21987,2015,ASSAULT,332.135468
2,174,South Eglinton-Davisville,21987,2016,ASSAULT,377.826965
3,174,South Eglinton-Davisville,21987,2017,ASSAULT,429.454285
4,174,South Eglinton-Davisville,21987,2018,ASSAULT,431.581635
...,...,...,...,...,...,...
14215,021,Humber Summit,13425,2019,THEFTOVER,166.590942
14216,021,Humber Summit,13425,2020,THEFTOVER,158.862244
14217,021,Humber Summit,13425,2021,THEFTOVER,220.030350
14218,021,Humber Summit,13425,2022,THEFTOVER,173.742264


In [31]:
# Saving csv file for all years
df_new.to_csv('../data/cleaned_files/neighbourhood_crime_split_2014_2023_clean.csv', index=False)

### Neighbourhood Median Income

In [32]:
nbhd_median_income = pd.read_csv("../data/source_files/neighbourhood-profiles-2021-158-model-inc-emply-educ-2020.csv")
nbhd_median_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 17 columns):
 #   Column                                                                                                                            Non-Null Count  Dtype  
---  ------                                                                                                                            --------------  -----  
 0   Neighbourhood Name                                                                                                                158 non-null    object 
 1   Neighbourhood Number                                                                                                              158 non-null    int64  
 2   TSNS 2020 Designation                                                                                                             158 non-null    object 
 3   Total - Income statistics for private households - 25% sample data                             

In [33]:
nbhd_median_income.rename(columns={'Neighbourhood Name':'NBHDName', 'Neighbourhood Number':'NBHDNo'}, inplace=True)

nbhd_median_income = nbhd_median_income[['NBHDNo', 'NBHDName', 'TSNS 2020 Designation', 'Total - Income statistics for private households - 25% sample data', 'Median total income of private household in 2020 ($)', 
                                         'Total - Income statistics for economic families in private households - 25% sample data', 'Median total income of economic family in 2020 ($)', 'Participation rate', 'Employment rate', 
                                         'Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data', 'No certificate, diploma or degree', 'No certificate, diploma or degree (%)',
                                         'High (secondary) school diploma or equivalency certificate', 'High (secondary) school diploma or equivalency certificate (%)', 'Postsecondary certificate, diploma or degree', 
                                         'Postsecondary certificate, diploma or degree (%)']]

# Convert 'NBHDNo' to string and add leading zeros
nbhd_median_income['NBHDNo'] = nbhd_median_income['NBHDNo'].apply(lambda x: str(x).zfill(3))

nbhd_median_income

,NBHDNo,NBHDName,TSNS 2020 Designation,Total - Income statistics for private households - 25% sample data,Median total income of private household in 2020 ($),Total - Income statistics for economic families in private households - 25% sample data,Median total income of economic family in 2020 ($),Participation rate,Employment rate,"Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data","No certificate, diploma or degree","No certificate, diploma or degree (%)",High (secondary) school diploma or equivalency certificate,High (secondary) school diploma or equivalency certificate (%),"Postsecondary certificate, diploma or degree","Postsecondary certificate, diploma or degree (%)"
0,001,West Humber-Clairville,Not an NIA or Emerging Neighbourhood,10700,92000,8130,104000,63.5,54.4,29000,5305,18.29,8450,29.14,15245,52.57
1,002,Mount Olive-Silverstone-Jamestown,Neighbourhood Improvement Area,9740,76500,7665,84000,56.0,46.1,25655,7130,27.79,7775,30.31,10755,41.92
2,003,Thistletown-Beaumond Heights,Neighbourhood Improvement Area,3245,86000,2470,100000,59.8,49.9,8350,1860,22.28,2375,28.44,4125,49.40
3,004,Rexdale-Kipling,Not an NIA or Emerging Neighbourhood,3945,77000,2685,99000,60.2,51.8,8805,1870,21.24,2720,30.89,4210,47.81
4,005,Elms-Old Rexdale,Neighbourhood Improvement Area,3190,82000,2450,95000,57.1,46.5,7745,1755,22.66,2630,33.96,3355,43.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,170,Yonge-Bay Corridor,Not an NIA or Emerging Neighbourhood,7465,68500,2810,100000,73.8,64.0,11675,315,2.70,1785,15.29,9580,82.06
154,171,Junction-Wallace Emerson,Not an NIA or Emerging Neighbourhood,10185,84000,6145,112000,69.8,60.0,20105,3980,19.80,4275,21.26,11850,58.94
155,172,Dovercourt Village,Not an NIA or Emerging Neighbourhood,5310,87000,3205,111000,67.7,58.4,11015,2065,18.75,2285,20.74,6665,60.51
156,173,North Toronto,Not an NIA or Emerging Neighbourhood,9430,70000,4040,96000,74.6,65.0,14570,795,5.46,2130,14.62,11645,79.92


In [34]:
# Saving csv file for all years
nbhd_median_income.to_csv('../data/cleaned_files/neighbourhood_median_income_2020_clean.csv', index=False)

### Neighbourhood Overview 2015-2019

In [35]:
NBHD_Overview = nbhd_median_income[['NBHDNo', 'NBHDName','TSNS 2020 Designation', 'Median total income of private household in 2020 ($)','Median total income of economic family in 2020 ($)', 'Participation rate', 'Employment rate',
                                   'No certificate, diploma or degree (%)', 'High (secondary) school diploma or equivalency certificate (%)', 'Postsecondary certificate, diploma or degree (%)']].copy()

branch_counts = branch_info_df.groupby('NBHDNo').size()
NBHD_Overview['No_of_Branches'] = NBHD_Overview['NBHDNo'].map(branch_counts)

NBHD_Overview = NBHD_Overview.drop_duplicates()
NBHD_Overview['No_of_Branches'] = NBHD_Overview['No_of_Branches'].fillna(0).astype(int)

wrkstn_counts = branch_info_df.groupby('NBHDNo')['Workstations'].sum()
NBHD_Overview['No_of_Workstations'] = NBHD_Overview['NBHDNo'].map(wrkstn_counts)
NBHD_Overview['No_of_Workstations'] = NBHD_Overview['No_of_Workstations'].fillna(0).astype(int)

program_count = branch_info_df.groupby('NBHDNo')['No_of_Programs'].sum()
NBHD_Overview['No_of_Programs'] = NBHD_Overview['NBHDNo'].map(program_count)
NBHD_Overview['No_of_Programs'] = NBHD_Overview['No_of_Programs'].fillna(0).astype(int)

NBHD_Overview

,NBHDNo,NBHDName,TSNS 2020 Designation,Median total income of private household in 2020 ($),Median total income of economic family in 2020 ($),Participation rate,Employment rate,"No certificate, diploma or degree (%)",High (secondary) school diploma or equivalency certificate (%),"Postsecondary certificate, diploma or degree (%)",No_of_Branches,No_of_Workstations,No_of_Programs
0,001,West Humber-Clairville,Not an NIA or Emerging Neighbourhood,92000,104000,63.5,54.4,18.29,29.14,52.57,2,13,0
1,002,Mount Olive-Silverstone-Jamestown,Neighbourhood Improvement Area,76500,84000,56.0,46.1,27.79,30.31,41.92,1,38,7
2,003,Thistletown-Beaumond Heights,Neighbourhood Improvement Area,86000,100000,59.8,49.9,22.28,28.44,49.40,0,0,0
3,004,Rexdale-Kipling,Not an NIA or Emerging Neighbourhood,77000,99000,60.2,51.8,21.24,30.89,47.81,1,6,0
4,005,Elms-Old Rexdale,Neighbourhood Improvement Area,82000,95000,57.1,46.5,22.66,33.96,43.32,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,170,Yonge-Bay Corridor,Not an NIA or Emerging Neighbourhood,68500,100000,73.8,64.0,2.70,15.29,82.06,1,9,0
154,171,Junction-Wallace Emerson,Not an NIA or Emerging Neighbourhood,84000,112000,69.8,60.0,19.80,21.26,58.94,1,3,0
155,172,Dovercourt Village,Not an NIA or Emerging Neighbourhood,87000,111000,67.7,58.4,18.75,20.74,60.51,0,0,0
156,173,North Toronto,Not an NIA or Emerging Neighbourhood,70000,96000,74.6,65.0,5.46,14.62,79.92,0,0,0


In [36]:
# Saving csv file for all years
NBHD_Overview.to_csv('../data/cleaned_files/NBHD_Overview_clean.csv', index=False)

In [37]:
nbhd_year_lvl_1 = nbhd_median_income[['NBHDNo', 'NBHDName']].copy()
nbhd_year_lvl_1

,NBHDNo,NBHDName
0,001,West Humber-Clairville
1,002,Mount Olive-Silverstone-Jamestown
2,003,Thistletown-Beaumond Heights
3,004,Rexdale-Kipling
4,005,Elms-Old Rexdale
...,...,...
153,170,Yonge-Bay Corridor
154,171,Junction-Wallace Emerson
155,172,Dovercourt Village
156,173,North Toronto


In [38]:
nbhd_yr_visits = branch_visits.groupby(['NBHDNo', 'Year'])['Visits'].sum().reset_index()

nbhd_year_lvl_1 = pd.merge(nbhd_year_lvl_1, nbhd_yr_visits, on='NBHDNo', how='left')

nbhd_year_lvl_1 = nbhd_year_lvl_1.fillna(0)

nbhd_year_lvl_1.rename(columns={'Visits': 'No_of_Visits'}, inplace=True)

nbhd_year_lvl_1 = nbhd_year_lvl_1.astype({'Year': int, 'No_of_Visits': int})

nbhd_year_lvl_1

,NBHDNo,NBHDName,Year,No_of_Visits
0,001,West Humber-Clairville,2012,248354
1,001,West Humber-Clairville,2013,253926
2,001,West Humber-Clairville,2014,195625
3,001,West Humber-Clairville,2015,144976
4,001,West Humber-Clairville,2016,172245
...,...,...,...,...
1029,171,Junction-Wallace Emerson,2021,8267
1030,171,Junction-Wallace Emerson,2022,24922
1031,172,Dovercourt Village,0,0
1032,173,North Toronto,0,0


In [39]:
nbhd_year_lvl_2 = nbhd_year_lvl_1.copy()

In [40]:
nbhd_yr_regs = branch_regs.groupby(['NBHDNo','Year'])['Registrations'].sum().reset_index()

nbhd_year_lvl_2 = pd.merge(nbhd_year_lvl_2, nbhd_yr_regs, on=['NBHDNo','Year'], how='outer')

nbhd_year_lvl_2 = nbhd_year_lvl_2.fillna(0)

nbhd_year_lvl_2.rename(columns={'Registrations': 'No_of_Registrations'}, inplace=True)

nbhd_year_lvl_2 = nbhd_year_lvl_2.astype({'No_of_Registrations': int})

In [41]:
nbhd_year_lvl_2

,NBHDNo,NBHDName,Year,No_of_Visits,No_of_Registrations
0,001,West Humber-Clairville,2012,248354.0,931
1,001,West Humber-Clairville,2013,253926.0,896
2,001,West Humber-Clairville,2014,195625.0,851
3,001,West Humber-Clairville,2015,144976.0,864
4,001,West Humber-Clairville,2016,172245.0,997
...,...,...,...,...,...
1038,096,0,2021,0.0,18
1039,099,0,2021,0.0,4
1040,099,0,2022,0.0,7
1041,115,0,2012,0.0,3


In [42]:
nbhd_year_lvl_3 = nbhd_year_lvl_2.copy()

In [43]:
nbhd_yr_circ = branch_circ.groupby(['NBHDNo','Year'])['Circulation'].sum().reset_index()

nbhd_year_lvl_3 = pd.merge(nbhd_year_lvl_3, nbhd_yr_circ, on=['NBHDNo','Year'], how='outer')

nbhd_year_lvl_3 = nbhd_year_lvl_3.fillna(0)

nbhd_year_lvl_3.rename(columns={'Circulation': 'Count_of_Circulation'}, inplace=True)

nbhd_year_lvl_3 = nbhd_year_lvl_3.astype({'Count_of_Circulation': int})

nbhd_year_lvl_3

,NBHDNo,NBHDName,Year,No_of_Visits,No_of_Registrations,Count_of_Circulation
0,001,West Humber-Clairville,2012,248354.0,931,197190
1,001,West Humber-Clairville,2013,253926.0,896,196768
2,001,West Humber-Clairville,2014,195625.0,851,180424
3,001,West Humber-Clairville,2015,144976.0,864,166526
4,001,West Humber-Clairville,2016,172245.0,997,151883
...,...,...,...,...,...,...
1038,096,0,2021,0.0,18,3768
1039,099,0,2021,0.0,4,11242
1040,099,0,2022,0.0,7,10191
1041,115,0,2012,0.0,3,9180


In [44]:
nbhd_year_lvl_4 = nbhd_year_lvl_3.copy()

In [45]:
nbhd_yr_session = branch_wrkstn_all.groupby(['NBHDNo','Year'])['Sessions'].sum().reset_index()

nbhd_year_lvl_4 = pd.merge(nbhd_year_lvl_4, nbhd_yr_session, on=['NBHDNo','Year'], how='outer')

nbhd_year_lvl_4 = nbhd_year_lvl_4.fillna(0)

nbhd_year_lvl_4.rename(columns={'Sessions': 'No_of_Sessions'}, inplace=True)

nbhd_year_lvl_4 = nbhd_year_lvl_4.astype({'No_of_Sessions': int})

nbhd_year_lvl_4

,NBHDNo,NBHDName,Year,No_of_Visits,No_of_Registrations,Count_of_Circulation,No_of_Sessions
0,001,West Humber-Clairville,2012,248354.0,931,197190,43737
1,001,West Humber-Clairville,2013,253926.0,896,196768,49738
2,001,West Humber-Clairville,2014,195625.0,851,180424,47113
3,001,West Humber-Clairville,2015,144976.0,864,166526,32900
4,001,West Humber-Clairville,2016,172245.0,997,151883,29488
...,...,...,...,...,...,...,...
1038,096,0,2021,0.0,18,3768,0
1039,099,0,2021,0.0,4,11242,0
1040,099,0,2022,0.0,7,10191,0
1041,115,0,2012,0.0,3,9180,0


In [46]:
nbhd_year_lvl_5 = nbhd_year_lvl_4.copy()

# Using isnull()
blank_rows = nbhd_year_lvl_5[nbhd_year_lvl_5['NBHDName'] == 0]
blank_rows

,NBHDNo,NBHDName,Year,No_of_Visits,No_of_Registrations,Count_of_Circulation,No_of_Sessions
1034,047,0,2013,0.0,26,186152,0
1035,091,0,2018,0.0,6,4239,0
1036,096,0,2019,0.0,17,14190,0
1037,096,0,2020,0.0,4,5174,0
1038,096,0,2021,0.0,18,3768,0
1039,099,0,2021,0.0,4,11242,0
1040,099,0,2022,0.0,7,10191,0
1041,115,0,2012,0.0,3,9180,0
1042,170,0,2021,0.0,28,10989,0


In [47]:
# Map 'NBHDName' from 'NBHD_Overview' to 'blank_rows' based on 'NBHDNo'
blank_rows['NBHDName'] = blank_rows['NBHDNo'].map(NBHD_Overview.set_index('NBHDNo')['NBHDName'])

blank_rows

C:\Users\Sonia\AppData\Local\Temp\ipykernel_19464\3636730837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blank_rows['NBHDName'] = blank_rows['NBHDNo'].map(NBHD_Overview.set_index('NBHDNo')['NBHDName'])


,NBHDNo,NBHDName,Year,No_of_Visits,No_of_Registrations,Count_of_Circulation,No_of_Sessions
1034,047,Don Valley Village,2013,0.0,26,186152,0
1035,091,Weston-Pelham Park,2018,0.0,6,4239,0
1036,096,Casa Loma,2019,0.0,17,14190,0
1037,096,Casa Loma,2020,0.0,4,5174,0
1038,096,Casa Loma,2021,0.0,18,3768,0
1039,099,Mount Pleasant East,2021,0.0,4,11242,0
1040,099,Mount Pleasant East,2022,0.0,7,10191,0
1041,115,Mount Dennis,2012,0.0,3,9180,0
1042,170,Yonge-Bay Corridor,2021,0.0,28,10989,0


In [48]:
# Select rows where 'NBHDName' is 0 in 'nbhd_year_lvl_5'
blank_rows = nbhd_year_lvl_5[nbhd_year_lvl_5['NBHDName'] == 0]

# Map 'NBHDName' from 'blank_rows' to 'nbhd_year_lvl_5' based on 'NBHDNo'
nbhd_year_lvl_5.loc[nbhd_year_lvl_5['NBHDName'] == 0, 'NBHDName'] = blank_rows['NBHDNo'].map(NBHD_Overview.set_index('NBHDNo')['NBHDName'])
nbhd_year_lvl_5

,NBHDNo,NBHDName,Year,No_of_Visits,No_of_Registrations,Count_of_Circulation,No_of_Sessions
0,001,West Humber-Clairville,2012,248354.0,931,197190,43737
1,001,West Humber-Clairville,2013,253926.0,896,196768,49738
2,001,West Humber-Clairville,2014,195625.0,851,180424,47113
3,001,West Humber-Clairville,2015,144976.0,864,166526,32900
4,001,West Humber-Clairville,2016,172245.0,997,151883,29488
...,...,...,...,...,...,...,...
1038,096,Casa Loma,2021,0.0,18,3768,0
1039,099,Mount Pleasant East,2021,0.0,4,11242,0
1040,099,Mount Pleasant East,2022,0.0,7,10191,0
1041,115,Mount Dennis,2012,0.0,3,9180,0


In [49]:
nbhd_overview_all_yr_lvl_stats = nbhd_year_lvl_5.copy()

In [50]:
# Saving csv file for all years
nbhd_overview_all_yr_lvl_stats.to_csv('../data/cleaned_files/nbhd_overview_all_yr_lvl_stats_clean.csv', index=False)

In [51]:
nbhd_overview_2015_2019_stats = nbhd_overview_all_yr_lvl_stats[nbhd_overview_all_yr_lvl_stats['Year'].isin([2015, 2016, 2017, 2018, 2019])]
nbhd_overview_2015_2019_stats

,NBHDNo,NBHDName,Year,No_of_Visits,No_of_Registrations,Count_of_Circulation,No_of_Sessions
3,001,West Humber-Clairville,2015,144976.0,864,166526,32900
4,001,West Humber-Clairville,2016,172245.0,997,151883,29488
5,001,West Humber-Clairville,2017,154296.0,1022,139851,26838
6,001,West Humber-Clairville,2018,134675.0,1192,122494,24800
7,001,West Humber-Clairville,2019,127903.0,1224,113892,23185
...,...,...,...,...,...,...,...
1025,171,Junction-Wallace Emerson,2017,53451.0,376,92619,8425
1026,171,Junction-Wallace Emerson,2018,55219.0,574,94768,5775
1027,171,Junction-Wallace Emerson,2019,52157.0,548,97542,5480
1035,091,Weston-Pelham Park,2018,0.0,6,4239,0


In [52]:
nbhd_overview_2015_2019_stats.to_csv('../data/cleaned_files/nbhd_overview_2015_2019_stats_clean.csv', index=False)

In [53]:
nbhd_overview_2015_2019_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 445 entries, 3 to 1036
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NBHDNo                445 non-null    object 
 1   NBHDName              445 non-null    object 
 2   Year                  445 non-null    int64  
 3   No_of_Visits          445 non-null    float64
 4   No_of_Registrations   445 non-null    int32  
 5   Count_of_Circulation  445 non-null    int32  
 6   No_of_Sessions        445 non-null    int32  
dtypes: float64(1), int32(3), int64(1), object(2)
memory usage: 22.6+ KB
